# Splits extraction using dedicated service - Python

### Overview
This sample requests **on-demand** splits by calling a dedicated data service.

### Inputs/outputs
Splits sample is designed to help you searching and requesting dividends for a given instrument.
* It takes instrument specification parameter(s) as per input filter and returns *Dividends*

### Services used
This sample uses *gRPC requests* in order to retrieve splits data from the hosted service. The queried endpoint in this script are:
* *SplitsService*: to directly retrieve splits data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.corporate_actions.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run splits queries

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.corporate_actions.v1.splits_pb2 as splits
import systemathics.apis.services.corporate_actions.v1.splits_pb2_grpc as splits_service

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve dividends

#### 3.1 Equity selection

In [ ]:
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Create and process request

In [ ]:
# create the splits request
request = splits.SplitsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
# open a gRPC channel
with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
    credentials = grpc.ssl_channel_credentials(f.read())
with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
    
    #instantiate the splits service
    service = splits_service.SplitsServiceStub(channel)
    
    # process the request
    response = service.Splits(
        request = request, 
        metadata = [('authorization', token)]
    ) 

#### 3.3 Retrieve splits

In [ ]:
# prepare the splits data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
old_shares = [d.old_shares for d in response.data]
new_shares = [d.new_shares for d in response.data]

# create pandas dataframe
d = {'Date': dates, 'New shares': new_shares, 'Old shares': old_shares}
df = pd.DataFrame(data=d)

In [ ]:
# display splits
df

#### 3.4 Add dates constraints

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 1, day = 1),
    end_date = date.Date(year = 2021, month = 3, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

In [ ]:
# create the splits request with dates constraints
request = splits.SplitsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints
)

In [ ]:
# open a gRPC channel
with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
    credentials = grpc.ssl_channel_credentials(f.read())
with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
    
    #instantiate the dividends service
    service = splits_service.SplitsServiceStub(channel)
    
    # process the request
    response = service.Splits(
        request = request, 
        metadata = [('authorization', token)]
    ) 

#### 3.5 Retrieve splits with date constraints

In [ ]:
# prepare the splits data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
old_shares = [d.old_shares for d in response.data]
new_shares = [d.new_shares for d in response.data]

# create pandas dataframe
d = {'Date': dates, 'New shares': new_shares, 'Old shares': old_shares}
df = pd.DataFrame(data=d)

In [ ]:
# display splits
df